Importação de bibliotecas (pacotes)

In [60]:
#!pip install --upgrade pandas # para atualizar a versão do pandas

In [61]:
import numpy as np
import pandas as pd
import datetime as dt
from babel.numbers import format_currency

Definindo as Hipóteses Atuariais e Características do Plano

In [62]:
# Definindo diretório de dados
#diretorio = r'C:\Users\helde\Desktop\Referências\TCC\Base de Dados'

# Importando o Banco de Tábuas Biométricas (Fonte: IBA)
banco_tabuas = pd.read_excel('https://github.com/helderbelo/tcc/blob/main/banco_tabuas.xlsx?raw=true', sheet_name='tabuas')

# Definindo as Hipóteses Atuariais e Características do Plano
mortalidade_geral_F = 'BR-EMSsb-v.2010-f' #'AT-2000 FEMALE'
mortalidade_geral_M = 'BR-EMSsb-v.2010-m' #'AT-2000 MALE'

#mortalidade_invalidos_F = 'WINKLEVOSS'
#mortalidade_invalidos_M = 'WINKLEVOSS'

entrada_invalidez = 'ALVARO VINDAS'

database = pd.to_datetime(dt.date(2022, 9, 30)) # DATA BASE DA AVALIAÇÃO (Ano, Mês, Dia)

idade_aposentadoria = 55        # Idade mínima de aposentadoria
tempo_patrocinadora = 15        # Tempo mínimo de vinculação com a patrocinadora
tempo_plano = 10                # Tempo mínimo de adesão ao plano

teto_salario_contribuicao = 15000   # Teto Salário Contribição / Participação (R$ 15.000,00)

contribuicao_ativo = 5 /100     # Contribuiçao Ativo (%)
contribuicao_patroc = 5 /100    # Contribuiçao Patrocinadora (%)

perc_beneficio_salario = 80 /100    # Valor do Benefício (% do salário)
perc_reversao_pensao = 60 /100    # Reversão de Pensão (% do benefício)

i = 6 /100      # Taxa de Juros (%)
v = 1/(1+i)     # Fator de Descapitalização

idade_maxima_rotatividade = 48      # Última idade que considera probabilidade de desligamento
percentual_rotatividade = 1/100     # % Rotatividade (desligamento)

crescimento_salarial = 2.01 /100 # % Crescimento Real dos Salários a.a.

perc_contr_ativo = 5/100    # Contribuiçao Ativo (%)
perc_contr_patroc = 5/100   # Contribuiçao Patrocinadora (%)

tx_carregamento = 15/100    # Taxa de Carregamento (% da contrbuição)
#Composição Grupo Familiar

per_casados_FEM = 80/100    # % de participantes femininas casadas
per_casados_MAS = 95/100    # % de participantes masculinos casados

dif_conjuge = 4 # Homem 4 anos a mais que a mulher



Tratando o DataFrame para construir uma tábua multidecremental

In [63]:
#======== Tábua Feminina ===============

tabua_feminina = pd.DataFrame(None)

tabua_feminina['qx'] = banco_tabuas[[mortalidade_geral_F]]          # definindo a tábua para a variável qx
tabua_feminina = tabua_feminina[tabua_feminina['qx'].notna()]       # excluindo as idades sem valores para qx (últimas idades pós idade ômega)

tabua_feminina.reset_index(names=['x'],inplace=True)                # utilizando a coluna índice para representar as idades 'x'

tabua_feminina['qxi'] =  0.0 #banco_tabuas[[mortalidade_invalidos_F]]
tabua_feminina['ix'] = banco_tabuas[[entrada_invalidez]].fillna(0)  # preenche os valores 'nan' com zeros

#======== Tábua Masculina ===============
# Repetindo o processo para a tábua masculina

tabua_masculina = pd.DataFrame(None)
tabua_masculina['qx'] = banco_tabuas[[mortalidade_geral_M]]
tabua_masculina = tabua_masculina[tabua_masculina['qx'].notna()] 

tabua_masculina.reset_index(names=['x'],inplace=True)

tabua_masculina['qxi'] = 0.0 #banco_tabuas[[mortalidade_invalidos_M]]
tabua_masculina['ix'] = banco_tabuas[[entrada_invalidez]].fillna(0)

#========== Idade ômega ============
w = max(tabua_feminina.x.values[-1],tabua_masculina.x.values[-1])

In [64]:
# Criando uma coluna para identificar cada tábua para ser utilizada quando juntar ambas em uma tábua única
tabua_masculina['id'] = 'M-'+tabua_masculina.x.astype(str)
tabua_feminina['id'] = 'F-'+tabua_masculina.x.astype(str)

In [65]:
tabuas=[tabua_feminina,tabua_masculina]

for tabua in tabuas:
    # Rotatividade a partir dos 18 anos até idade máxima considerada na hipótese
    tabua.loc[18:idade_maxima_rotatividade,'wx'] = (np.array([percentual_rotatividade]*(idade_maxima_rotatividade-17))) 
    
    # Preenchendo as demais idades com zeros
    tabua['wx'] = tabua['wx'].fillna(0.0)
    
    # Decrementos sem hipóteses definidas nesse exemplo
    tabua['zx'] = 0.0
    tabua['qr'] = 0.0

    # px = 1 - qx
    tabua['px'] = 1-tabua.qx
    
    # Probabilidades em ambiente de 4 decrementos
    tabua['qx_ai'] = tabua.ix*(1 - 1/2*(tabua.qx + tabua.wx + tabua.qr) + 1/3*(tabua.qx * tabua.wx + tabua.qx * tabua.qr + tabua.wx * tabua.qr) - 1/4*(tabua.qx * tabua.wx * tabua.qr))
    tabua['qx_aw'] = tabua.wx*(1 - 1/2*(tabua.qx + tabua.ix + tabua.qr) + 1/3*(tabua.qx * tabua.ix + tabua.qx * tabua.qr + tabua.ix * tabua.qr) - 1/4*(tabua.qx * tabua.ix * tabua.qr))
    tabua['qx_ar'] = tabua.qr*(1 - 1/2*(tabua.qx + tabua.ix + tabua.wx) + 1/3*(tabua.qx * tabua.ix + tabua.qx * tabua.wx + tabua.ix * tabua.wx) - 1/4*(tabua.qx * tabua.ix * tabua.wx))
    tabua['qx_aa'] = tabua.qx*(1 - 1/2*(tabua.ix + tabua.wx + tabua.qr) + 1/3*(tabua.ix * tabua.wx + tabua.ix * tabua.qr + tabua.wx * tabua.qr) - 1/4*(tabua.ix * tabua.wx * tabua.qr))
    
    tabua['px_ai'] = 1 - tabua.qx_ai
    tabua['px_aw'] = 1 - tabua.qx_aw
    tabua['px_ar'] = 1 - tabua.qx_ar
    
    # px_aa = probabilidade do participante permanecer ativo = 1 - qx's
    tabua['px_aa'] = 1 - (tabua.qx_aa + tabua.qx_ai + tabua.qx_ar + tabua.qx_aw)

    # Criando as colunas lx's inicialmente com valor zero
    tabua['lx'] = tabua['lx_aa'] = tabua['lx_i'] = tabua['lxai'] = tabua['lx_ii'] = 0.0
    
    # Atribuindo a raiz da tábua na idade zero
    tabua.loc[0,'lx'] = tabua.loc[0,'lx_aa'] = tabua.loc[0,'lx_i'] = 100000

    # Calculando os lx a partir da idade 1 com base em lx-1 - qx-1 * lx-1
    for i in tabua.x[1:]:
        tabua.loc[i,'lx'] = tabua.lx[i-1] - tabua.qx[i-1] * tabua.lx[i-1]  
        tabua.loc[i,'lx_i'] = tabua.lx_i[i-1] - tabua.qxi[i-1] * tabua.lx_i[i-1]
        tabua.loc[i,'lx_aa'] = tabua.lx_aa[i-1] * tabua.px_aa[i-1]
        tabua.loc[i,'lx_ii'] = tabua.lx_aa[i-1] * tabua.px_ai[i-1]

    tabua['dx'] = tabua.qx * tabua.lx
  
#tabua_feminina

In [66]:
# Para construir os valores das rendas conjuntas 
tabua_feminina['lxy'] = tabua_feminina.lx * tabua_masculina.lx.shift(-dif_conjuge) # No caso da tábua feminina, está pegando o lx da tábua do cônjuge deslocado a diferença de idade para mais
tabua_masculina['lxy'] = tabua_feminina.lx.shift(dif_conjuge) * tabua_masculina.lx

tabuas=[tabua_feminina,tabua_masculina]

def somatorio(coluna):
    somatorio = coluna.sum() - (np.cumsum(coluna.shift(1).fillna(0))) # Soma total - Soma acumulada de x0 a x-1
    return somatorio

for tabua in tabuas:
    
    Dx = tabua['Dx'] = tabua.lx * v ** tabua.x
    Dx_aa = tabua['Dx_aa'] = tabua.lx_aa * v ** tabua.x
            
    Nx = tabua['Nx'] = somatorio(Dx)
    Nx_aa = tabua['Nx_aa'] = somatorio(Dx_aa)
 
    tabua['Sx'] = somatorio(Nx)
    
    äx = tabua['äx'] = Nx/Dx
    äx_12 = tabua['äx_12'] = äx - 11/24
    äx_aa = tabua['äx_aa'] = Nx_aa/Dx_aa

    tabua['Dxy'] = tabua.lxy * v ** tabua.x
    tabua['Nxy'] = somatorio(tabua.Dxy)
    tabua['äxy'] = tabua.Nxy / tabua.Dxy
    tabua['äxy_12'] = np.where(tabua.äxy > 11/24, tabua.äxy - 11/24, 0)

In [67]:
#Importando a base de beneficiários
ativos = pd.read_excel('https://github.com/helderbelo/tcc/blob/main/base-bd.xlsx?raw=true', sheet_name='ativos')
aposentados = pd.read_excel('https://github.com/helderbelo/tcc/blob/main/base-bd.xlsx?raw=true', sheet_name='aposentados')
pensionistas = pd.read_excel('https://github.com/helderbelo/tcc/blob/main/base-bd.xlsx?raw=true', sheet_name='pensionistas')

In [68]:
# Base maior para teste de performance com 1 milhão de registros
#ativos = pd.read_csv('https://github.com/helderbelo/tcc/blob/main/base_ativos_teste.csv?raw=true')

# Base com 200.000 registros da base maior
#ativos = ativos.iloc[:200000,:]

In [69]:
colunas_datas=['DT NASC','DT ADMISSÃO','DT ADESÃO']

# Transformando as colunas de datas em formato de data reconhecível pelo python (caso precise)
for i in colunas_datas:
    ativos[i] = pd.to_datetime(ativos[i])

# Idade de Atual na Data Base dos Cálculos
ativos['IDADE_x'] = (database - ativos['DT NASC']).astype('timedelta64[Y]').astype(int)                

# Idade de Adesão
ativos['IDADE_e'] = (ativos['DT ADESÃO'] - ativos['DT NASC']).astype('timedelta64[Y]').astype(int)     

# Tempo na Patrocinadora na Data Base em anos
ativos['TEMP_PATROC'] = (database - ativos['DT ADMISSÃO']).astype('timedelta64[Y]').astype(int)         

# Tempo no Plano na Data Base em anos
ativos['TEMP_PLANO'] = (database - ativos['DT ADESÃO']).astype('timedelta64[Y]').astype(int)            

# Tempo para estar elegível para aposentadoria em anos
ativos['TEMP_APOSENT'] = np.maximum(0,
                                    np.maximum(idade_aposentadoria - ativos['IDADE_x'],    # Critério de idade
                                    np.maximum(tempo_plano - ativos['TEMP_PLANO'],          # Critério de Tempo de Adesão ao Plano
                                    tempo_patrocinadora - ativos['TEMP_PATROC'])))          # Critério de Tempo na Patrocinadora

# Idade na elegibilidade  para aposentadoria em anos
ativos['IDADE_APOSENT'] = ativos['IDADE_x'] + ativos['TEMP_APOSENT']

# Salário de Participação = Mínimo entre Teto de Contribuição ou Salário
ativos['SALARIO_PARTICIPACAO'] = np.minimum(teto_salario_contribuicao , ativos['SALARIO'])

# Salário na Aposentadoria = Mínimo entre Teto de Contribuição ou Salário Projetado
ativos['SALARIO_APOSENT'] = np.minimum(teto_salario_contribuicao , ativos['SALARIO'] * (1+crescimento_salarial) ** ativos['TEMP_APOSENT'])

# Benefício na Aposentadoria = Mínimo entre Teto de Contribuição ou Salário Projetado x % do salário para o benefício
ativos['BENEFICIO_z'] = np.minimum(teto_salario_contribuicao , ativos['SALARIO_APOSENT'] * perc_beneficio_salario)


In [70]:
# Sexo do conjuge com base no sexo oposto
ativos['SEXO_CONJUGE'] = np.where(ativos.SEXO == 'M', 'F', 'M')

# Idade do conjuge com base nas premissas
ativos['IDADE_CONJUGE'] = np.where(ativos.SEXO == 'M',ativos.IDADE_x - dif_conjuge,ativos.IDADE_x + dif_conjuge)

#ativos

In [71]:
# Unindo as duas tábuas de mortalidade (inserindo novas linhas)
tabua_unica = pd.concat([tabua_masculina,tabua_feminina])
tabua_unica.tail(10)

,x,qx,qxi,ix,id,wx,zx,qr,px,qx_ai,...,Nx,Nx_aa,Sx,äx,äx_12,äx_aa,Dxy,Nxy,äxy,äxy_12
107,107,0.47011,0.0,0.0,F-107,0.0,0.0,0.0,0.52989,0.0,...,2.313243e+00,1.766309e-01,4.096176e+00,1.868490,1.410157,1.868490,19.596973,23.119537,1.179750,0.721417
108,108,0.51562,0.0,0.0,F-108,0.0,0.0,0.0,0.48438,0.0,...,1.075215e+00,8.209954e-02,1.782932e+00,1.737341,1.279008,1.737341,3.131436,3.522552,1.124900,0.666567
109,109,0.56553,0.0,0.0,F-109,0.0,0.0,0.0,0.43447,0.0,...,4.563298e-01,3.484368e-02,7.077169e-01,1.613571,1.155238,1.613571,0.363132,0.391113,1.077056,0.618723
110,110,0.62029,0.0,0.0,F-110,0.0,0.0,0.0,0.37971,0.0,...,1.735224e-01,1.324954e-02,2.513871e-01,1.496963,1.038629,1.496963,0.027016,0.027985,1.035859,0.577526
111,111,0.68035,0.0,0.0,F-111,0.0,0.0,0.0,0.31965,0.0,...,5.760608e-02,4.398591e-03,7.786472e-02,1.387323,0.928989,1.387323,0.000989,0.000977,0.986994,0.528660
112,112,0.74623,0.0,0.0,F-112,0.0,0.0,0.0,0.25377,0.0,...,1.608288e-02,1.228030e-03,2.025864e-02,1.284412,0.826078,1.284411,0.000005,0.000000,0.000000,0.000000
113,113,0.81849,0.0,0.0,F-113,0.0,0.0,0.0,0.18151,0.0,...,3.561286e-03,2.719266e-04,4.175756e-03,1.187990,0.729657,1.187989,NaN,0.000000,NaN,0.000000
114,114,0.89776,0.0,0.0,F-114,0.0,0.0,0.0,0.10224,0.0,...,5.635459e-04,4.303013e-05,6.144680e-04,1.097844,0.639510,1.097839,NaN,0.000000,NaN,0.000000
115,115,0.98471,0.0,0.0,F-115,0.0,0.0,0.0,0.01529,0.0,...,5.022530e-05,3.834721e-06,5.092099e-05,1.014422,0.556089,1.014343,NaN,0.000000,NaN,0.000000
116,116,1.00000,0.0,0.0,F-116,0.0,0.0,0.0,0.00000,0.0,...,7.140916e-07,5.424954e-08,6.966293e-07,0.999882,0.541548,0.994822,NaN,0.000000,NaN,0.000000


In [72]:
#Trazendo os valores das funções de comutação com base no sexo e idade de aposentadoria

ativos['id'] =  ativos.SEXO+"-"+ativos.IDADE_APOSENT.astype(str)    # Criando uma chave única para dar match na tábua multidecremental unificada
ativos = ativos.merge(tabua_unica[['id','Dx_aa','äx_12','äxy_12']], on='id',how='left') # Unindo a base de ativos com os valores das funções de comutação
ativos.rename(columns={'äx_12':'äz_12','Dx_aa':'Dz_aa',},inplace=True)  # Renomeando a coluna para Dz_aa e äz_12 (Renda antecipada vitalícia e fracionada a partir da data de aposentadoria)
ativos.drop(columns='id',inplace=True)  # Excluindo a coluna de chave única
#ativos

In [73]:
#Trazendo os valores das funções de comutação com base no sexo e idade de atual

ativos['id'] =  ativos.SEXO+"-"+ativos.IDADE_x.astype(str)  # Criando uma chave única para dar match na tábua multidecremental unificada, dessa vez com a idade atual
ativos = ativos.merge(tabua_unica[['id','qx_ai','qx_aa','Dx_aa']], on='id',how='left')
ativos.drop(columns='id',inplace=True)
#ativos

In [74]:
#Trazendo os valores das funções de comutação com base no sexo do cônjuge e idade de cônjuge na aposentadoria do beneficiário

ativos['id'] =  np.where(ativos.SEXO_CONJUGE == 'M', #Se o sexo do cônjuge = Masculino, adicionará/subtrairá idade do aposentado conforme premissa adotada
                         ativos.SEXO_CONJUGE+"-"+(ativos.IDADE_APOSENT+dif_conjuge).astype(str),    # Caso que a condição é verdadeira
                         ativos.SEXO_CONJUGE+"-"+(ativos.IDADE_APOSENT-dif_conjuge).astype(str))    # Caso que a condição não é verdadeira

ativos = ativos.merge(tabua_unica[['id','äx_12']], on='id',how='left')
ativos.rename(columns={'äx_12':'äy_12'},inplace=True) # Renomeando a coluna para äy_12 (Renda antecipada vitalícia e fracionada para o cônjuge a partir da data de aposentadoria do beneficiário)
ativos.drop(columns='id',inplace=True)
#ativos

In [75]:
# Valor Atual dos Benefícios Futuros de Aposentadoria
# 13 pagamentos mensais x Valor do Benefício x Valor da Renda Vitalícia fracionada x nEx (Data de Aposentadoria / Data Atual)
ativos['VABF_APO'] = 13 * ativos['BENEFICIO_z'] * ativos['äz_12'] * ativos.Dz_aa/ativos.Dx_aa


# Valor Atual dos Benefícios Futuros Pecúlio por Invalidez 
# Maior valor entre o valor do Resgate ou 15x o valor do salário de participação x decremento de invalidez
ativos['VABF_INV'] =  np.maximum(ativos['RESGATE'], 15 * ativos['SALARIO_PARTICIPACAO']) * ativos['qx_ai']

# Valor Atual dos Benefícios Futuros Pecúlio por Morte
# Maior valor entre o valor do Resgate ou 15x o valor do salário de participação x decremento de morte
ativos['VABF_PEC'] =  np.maximum(ativos['RESGATE'], 15 * ativos['SALARIO_PARTICIPACAO']) * ativos['qx_aa']


# Valor Atual dos Benefícios Futuros de Aposentadoria com Reversão em Pensão
# 13 pagamentos mensais x Valor do Benefício x Diferença Valor da Renda Vitalícia fracionada do Cônjuge e da Renda Conjunta x nEx
ativos['VABF_REV_PENS'] = 13 * perc_reversao_pensao * ativos.BENEFICIO_z * (ativos.äy_12-ativos.äxy_12) * ativos.Dz_aa/ativos.Dx_aa

In [76]:
# Variável para armazenar os anos t de zero à idade ômega
fluxo = list(np.arange(0,1+max(ativos.TEMP_APOSENT)))

In [77]:
# Cálculo do nEx nos anos futuros com base na idade atual para ser utilizado no cálculo do Valor Atual dos Salários Futuros

# Criando uma cópia da base apenas com as colunas desejáveis
nEx_aa = ativos[['MATRICULA','SEXO','IDADE_x','IDADE_APOSENT','SALARIO']].copy()

for t in fluxo:
    nEx_aa['id'] = np.where(nEx_aa.IDADE_x+t <= nEx_aa.IDADE_APOSENT,   # Condição para trazer o valor do nEx apenas até a data de aposentadoria
                            nEx_aa.SEXO+"-"+(nEx_aa.IDADE_x+t).astype(str), # Criando chave única para cada t ano 
                            0.00)   # Valor zero a partir da data de aposentadoria
    
    nEx_aa = nEx_aa.merge(tabua_unica[['id','Dx_aa']], on='id',how='left')
    nEx_aa.rename(columns={'Dx_aa':t},inplace=True)
    
    nEx_aa[t] = nEx_aa[t] / ativos.Dx_aa # nEx = Dx+n / Dx

nEx_aa.drop(columns='id',inplace=True)
nEx_aa.head(10)


,MATRICULA,SEXO,IDADE_x,IDADE_APOSENT,SALARIO,0,1,2,3,4,...,16,17,18,19,20,21,22,23,24,25
0,1,M,59,59,18844.5,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,M,67,67,20604.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,M,59,59,12665.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,M,68,68,9401.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,F,54,55,5652.5,1.0,0.938881,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,M,52,55,19764.0,1.0,0.938101,0.879567,0.824195,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,M,60,60,6375.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,M,57,57,15376.5,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,M,44,55,20247.0,1.0,0.931202,0.866949,0.806930,0.750861,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,M,43,55,18445.0,1.0,0.931382,0.867305,0.807461,0.751560,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
# Apenas visualização dos dados
nEx_aa.iloc[:,5:]

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,0.938881,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.0,0.931000,0.866547,0.806336,0.750077,0.704554,0.661538,0.620886,0.582453,0.546110,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,1.0,0.932888,0.870224,0.811723,0.757100,0.706099,0.658478,0.614008,0.572475,0.533685,...,0.331558,0.311677,0.292883,0.275110,0.258295,NaN,NaN,NaN,NaN,NaN
97,1.0,0.932149,0.868814,0.809687,0.754490,0.702959,0.654839,0.609905,0.567945,0.528756,...,0.330801,0.310160,0.290635,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,1.0,0.933002,0.870463,0.812077,0.757577,0.706690,0.659182,0.614824,0.573407,0.534735,...,0.326829,0.304416,0.286333,0.269251,0.253106,0.237843,0.22341,0.209756,NaN,NaN


In [79]:
# Criando outro dataframe com o salário projetado com base nas premissas

vsf = ativos[['MATRICULA','SEXO','IDADE_x','IDADE_APOSENT','SALARIO']].copy()
vsf = vsf.reindex(columns=['MATRICULA','SEXO','IDADE_x','IDADE_APOSENT','SALARIO'] + fluxo)

for t in fluxo:
    
    vsf.loc[:,t] = (np.where(t + vsf['IDADE_x'] <= vsf['IDADE_APOSENT'],  
                                    np.minimum( teto_salario_contribuicao , 
                                               vsf['SALARIO']*(1+crescimento_salarial) ** t), 
                                               0.0))
            
vsf.head(10)


,MATRICULA,SEXO,IDADE_x,IDADE_APOSENT,SALARIO,0,1,2,3,4,...,16,17,18,19,20,21,22,23,24,25
0,1,M,59,59,18844.5,15000.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,M,67,67,20604.0,15000.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,M,59,59,12665.0,12665.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,M,68,68,9401.0,9401.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,F,54,55,5652.5,5652.5,5766.11525,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,M,52,55,19764.0,15000.0,15000.00000,15000.0,15000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,7,M,60,60,6375.0,6375.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,8,M,57,57,15376.5,15000.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,9,M,44,55,20247.0,15000.0,15000.00000,15000.0,15000.0,15000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,10,M,43,55,18445.0,15000.0,15000.00000,15000.0,15000.0,15000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [80]:
# Valor Atual dos Salários Futuros = Salários Projetado x Desconto Atuarial (nEX)

vsf.iloc[:,5:] = vsf.iloc[:,5:] * nEx_aa.iloc[:,5:]

vsf.iloc[:,5:]

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,15000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12665.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9401.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5652.5,5413.694675,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5865.0,5570.067696,5288.657866,5020.097172,4763.703731,4564.528587,4371.989282,4185.804555,4005.632257,3831.185785,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,5227.5,4974.690848,4733.809490,4504.330517,4285.667126,4077.304601,3878.750154,3689.499246,3509.074802,3337.055970,...,2383.072718,2285.203851,2190.570615,2098.997872,2010.319965,NaN,NaN,NaN,NaN,NaN
97,1632.0,1551.845199,1475.476487,1402.701683,1333.352136,1267.254845,1204.233954,1144.146578,1086.846178,1032.192120,...,742.285781,709.959016,678.636226,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,4777.0,4546.537139,4327.041264,4117.948474,3918.798773,3729.045158,3548.273740,3376.022267,3211.885272,3055.474563,...,2146.643615,2039.621100,1957.027178,1877.258854,1800.162929,1725.615863,1653.479691,1583.623918,NaN,NaN


In [81]:
# Valor Atual dos Salários Futuros está pegando a soma de todas as colunas para cada linha
ativos['VSF'] = vsf.iloc[:,5:].sum(axis=1)

# Valor Atual das Contribuições Futuras = (% de contribuição ativo + patrocinadora) x (1-taxa de carregamento) x Valor Atual dos Salários Futuros
ativos['VACF'] = 13 * (perc_contr_ativo + perc_contr_patroc) * (1 - tx_carregamento) * ativos['VSF']

# Custo Normal pelo Médoto Crédito Unitário = Soma dos Valores Atuais dos Benefícios Futuros / diferença entre idade de aposentadoria e idade de adesão
ativos['CN'] = (ativos.VABF_APO + ativos.VABF_INV + ativos.VABF_PEC + ativos.VABF_REV_PENS) / (ativos.IDADE_APOSENT - ativos.IDADE_e)

# Provisões Matemáticas dos Benefícios a Conceder Método Crédito Unitário = Custo Normal x diferença entre idade de aposentadoria e idade de adesão
ativos['PMBaC_UC'] = ativos.CN * (ativos.IDADE_x-ativos.IDADE_e)

# Custo Normal de contribução anual ao plano
ativos['CN_CONTRIB_PLANO'] = 13 * ativos.SALARIO_PARTICIPACAO * (perc_contr_ativo + perc_contr_patroc) * (1 - tx_carregamento)
ativos

,MATRICULA,DT NASC,DT ADMISSÃO,DT ADESÃO,SEXO,SALARIO,RESGATE,IDADE_x,IDADE_e,TEMP_PATROC,...,äy_12,VABF_APO,VABF_INV,VABF_PEC,VABF_REV_PENS,VSF,VACF,CN,PMBaC_UC,CN_CONTRIB_PLANO
0,1,1962-10-15,1980-02-15,1983-03-15,M,18844.5,289620.5,59,20,42,...,14.201704,1.976874e+06,1413.418005,1742.137273,212977.821948,15000.000000,16575.000000,56230.959607,2.193007e+06,16575.0000
1,2,1955-02-17,1981-05-15,1981-06-06,M,20604.0,453101.0,67,26,41,...,12.846677,1.708770e+06,5893.857904,5446.647217,262313.390837,15000.000000,16575.000000,48351.807269,1.982424e+06,16575.0000
2,3,1963-05-17,1983-11-18,1983-11-18,M,12665.0,177650.0,59,20,38,...,14.201704,1.669141e+06,927.123893,1142.745518,179824.274331,12665.000000,13994.825000,47462.430713,1.851035e+06,13994.8250
3,4,1954-03-12,1986-02-22,1986-02-22,M,9401.0,178211.0,68,31,36,...,12.649337,1.047468e+06,2628.396097,2340.228910,167851.446091,9401.000000,10388.105000,32980.771877,1.220289e+06,10388.1050
4,5,1968-02-05,1990-03-18,1990-03-18,F,5652.5,44591.0,54,22,32,...,12.672270,7.995904e+05,233.945379,171.880929,25199.936682,11066.194675,12228.145116,25005.943500,8.001902e+05,6246.0125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,1977-02-07,2016-02-15,2016-02-21,M,5865.0,47549.0,45,39,6,...,14.735733,5.099378e+05,102.663643,174.967656,45771.417221,51128.824051,56497.350576,34749.177416,2.084951e+05,6480.8250
96,97,1987-02-14,2012-08-24,2017-05-12,F,5227.5,43375.5,35,30,10,...,12.672270,2.969197e+05,53.119453,36.657141,9357.737900,70041.474451,77395.829268,12254.687776,6.127344e+04,5776.3875
97,98,1985-08-22,2018-05-25,2018-07-16,M,1632.0,6664.0,37,32,4,...,14.735733,9.449684e+04,17.818994,29.461927,8481.924953,20485.719658,22636.720222,4479.393087,2.239697e+04,1803.3600
98,99,1989-11-15,2019-03-22,2019-03-22,F,4777.0,7089.0,32,29,3,...,12.672270,2.338976e+05,44.765403,28.509750,7371.531803,68825.553594,76052.236722,9282.402024,2.784721e+04,5278.5850


In [82]:
print(
'PMBaC',

'\nVABF - Aposentadoria:',  format_currency(ativos.VABF_APO.sum(), 'BRL', locale='pt_BR'),
'\nVABF - Reversão em Pensão:', format_currency(ativos.VABF_REV_PENS.sum(), 'BRL', locale='pt_BR'),
'\nVABF - Invalidez:', format_currency(ativos.VABF_INV.sum(), 'BRL', locale='pt_BR'),
'\nVABF - Pecúlio:', format_currency(ativos.VABF_PEC.sum(), 'BRL', locale='pt_BR'),
'\n---------------------------------------------',
'\nTOTAL VABF:',  format_currency(ativos.VABF_APO.sum()+ativos.VABF_REV_PENS.sum()+ativos.VABF_INV.sum()+ativos.VABF_PEC.sum(), 'BRL', locale='pt_BR')


)

PMBaC 
VABF - Aposentadoria: R$ 84.104.771,18 
VABF - Reversão em Pensão: R$ 6.570.971,81 
VABF - Invalidez: R$ 37.920,59 
VABF - Pecúlio: R$ 45.963,02 
--------------------------------------------- 
TOTAL VABF: R$ 90.759.626,61


In [83]:
aposentados['IDADE_x'] = (database - aposentados['DT NASC']).astype('timedelta64[Y]').astype(int)     

# Sexo do conjuge com base no sexo oposto
aposentados['SEXO_CONJUGE'] = np.where(aposentados.SEXO == 'M', 'F', 'M')

# Idade do conjuge com base nas premissas
aposentados['IDADE_CONJUGE'] = np.where(aposentados.SEXO == 'M',aposentados.IDADE_x-dif_conjuge,aposentados.IDADE_x+dif_conjuge)

aposentados['id'] =  np.where(aposentados.SEXO_CONJUGE == 'M',      #Se o sexo do cônjuge = Masculino, adicionará/subtrairá idade do aposentado conforme premissa 
                              aposentados.SEXO_CONJUGE+"-"+(aposentados.IDADE_x+dif_conjuge).astype(str),
                              aposentados.SEXO_CONJUGE+"-"+(aposentados.IDADE_x-dif_conjuge).astype(str))

aposentados = aposentados.merge(tabua_unica[['id','äx_12']], on='id',how='left')
aposentados.rename(columns={'äx_12':'äy_12'},inplace=True)
aposentados.drop(columns='id',inplace=True)

#==================================================

aposentados['id'] =  aposentados.SEXO+"-"+aposentados.IDADE_x.astype(str)

aposentados = aposentados.merge(tabua_unica[['id','äx_12','äxy_12']], on='id',how='left')
aposentados.drop(columns='id',inplace=True)

aposentados['PMBC_APO'] = 13 * aposentados.BENEFICIO * aposentados['äx_12']

aposentados['PMBC_REV_PENS'] =  13 * perc_reversao_pensao * aposentados.BENEFICIO * (aposentados.äy_12 - aposentados.äxy_12)

aposentados

,MATRICULA,DT NASC,DIB,SEXO,BENEFICIO,IDADE_x,SEXO_CONJUGE,IDADE_CONJUGE,äy_12,äx_12,äxy_12,PMBC_APO,PMBC_REV_PENS
0,101,1946-11-03,2002-12-31,M,6570,75,F,71,11.074630,8.906253,7.904586,7.606831e+05,162452.063202
1,102,1934-05-28,1992-08-08,M,4635,88,F,84,7.158827,5.323788,4.317610,3.207848e+05,102718.540784
2,103,1942-12-22,1997-12-22,F,2583,79,M,83,6.688065,8.817651,5.670953,2.960879e+05,20492.152957
3,104,1949-08-26,2019-01-03,M,3987,73,F,69,11.561345,9.442166,8.456206,4.893969e+05,96565.466180
4,105,1950-05-28,1996-05-21,M,1323,72,F,68,11.793106,9.704952,8.728466,1.669155e+05,31625.244736
5,106,1939-12-27,2017-08-02,M,12465,82,F,78,9.128428,6.966683,5.948864,1.128916e+06,309139.449703
6,107,1952-11-07,2012-02-02,F,9216,69,M,73,9.442166,11.561345,8.456206,1.385142e+06,70875.514577
7,108,1950-06-19,1997-11-13,F,1044,72,M,76,8.633888,10.820069,7.626419,1.468500e+05,8204.021402
8,109,1962-10-17,2008-10-01,M,1386,59,F,55,14.201704,12.672270,11.926300,2.283290e+05,24598.938435
9,110,1953-01-17,2014-10-21,M,14256,69,F,65,12.445607,10.468429,9.528232,1.940093e+06,324402.743727


In [84]:
pensionistas['IDADE_x'] = (database - pensionistas['DT NASC']).astype('timedelta64[Y]').astype(int)     

pensionistas['id'] =  pensionistas.SEXO+"-"+pensionistas.IDADE_x.astype(str)

pensionistas = pensionistas.merge(tabua_unica[['id','äx_12']], on='id',how='left')
pensionistas.drop(columns='id',inplace=True)

pensionistas['PMBC_PENS'] = 13 * pensionistas.BENEFICIO * pensionistas['äx_12']

pensionistas

,MATRICULA,DT NASC,DIB,SEXO,BENEFICIO,IDADE_x,äx_12,PMBC_PENS
0,201,1949-05-21,2002-10-26,F,330,73,10.558175,4.529457e+04
1,202,1957-06-24,2020-03-27,F,880,65,12.445607,1.423777e+05
2,203,1953-05-25,2012-05-20,F,1120,69,11.561345,1.683332e+05
3,204,1940-03-24,1998-01-20,F,680,82,7.837518,6.928366e+04
4,205,1979-09-05,2005-10-18,M,670,43,14.996803,1.306222e+05
5,206,1949-09-27,2015-06-28,F,1550,73,10.558175,2.127472e+05
6,207,1941-05-26,2016-07-23,F,3270,81,8.171033,3.473506e+05
7,208,1940-10-16,1981-07-27,F,390,81,8.171033,4.142714e+04
8,209,1934-07-16,2021-09-06,F,7140,88,5.791965,5.376101e+05
9,210,1950-08-30,2008-05-11,F,2220,72,10.820069,3.122672e+05


In [85]:
print(
'PMBC',

'\nAposentadorias:',  format_currency(aposentados.PMBC_APO.sum(), 'BRL', locale='pt_BR'),
'\nReversão em Pensão:', format_currency(aposentados.PMBC_REV_PENS.sum(), 'BRL', locale='pt_BR'),
'\nPensões Concedidas:', format_currency(pensionistas.PMBC_PENS.sum(), 'BRL', locale='pt_BR'),
'\n---------------------------------------------',
'\nTOTAL PMBC:',  format_currency(aposentados.PMBC_APO.sum()+aposentados.PMBC_REV_PENS.sum()+pensionistas.PMBC_PENS.sum(), 'BRL', locale='pt_BR'),


'\n\nPMBaC',

'\nVABF - Aposentadoria:',  format_currency(ativos.VABF_APO.sum(), 'BRL', locale='pt_BR'),
'\nVABF - Reversão em Pensão:', format_currency(ativos.VABF_REV_PENS.sum(), 'BRL', locale='pt_BR'),
'\nVABF - Invalidez:', format_currency(ativos.VABF_INV.sum(), 'BRL', locale='pt_BR'),
'\nVABF - Pecúlio:', format_currency(ativos.VABF_PEC.sum(), 'BRL', locale='pt_BR'),
'\n---------------------------------------------',
'\nTOTAL VABF:',  format_currency(ativos.VABF_APO.sum()+ativos.VABF_REV_PENS.sum()+ativos.VABF_INV.sum()+ativos.VABF_PEC.sum(), 'BRL', locale='pt_BR'),

'\n\nVACF (13x valor mensal):', format_currency(ativos.VACF.sum(), 'BRL', locale='pt_BR'),
'\nVASF (13x valor mensal):', format_currency(ativos.VSF.sum() * 13, 'BRL', locale='pt_BR'),

'\n\nPMBaC Crédito Unitário:',  format_currency(ativos.PMBaC_UC.sum(), 'BRL', locale='pt_BR'),
'\nPMBaC Custo Agregado:',  format_currency((ativos.VABF_APO.sum()+ativos.VABF_REV_PENS.sum()+ativos.VABF_INV.sum()+ativos.VABF_PEC.sum())-ativos.VACF.sum(), 'BRL', locale='pt_BR'),
)

PMBC 
Aposentadorias: R$ 65.661.235,77 
Reversão em Pensão: R$ 10.525.402,04 
Pensões Concedidas: R$ 13.001.345,08 
--------------------------------------------- 
TOTAL PMBC: R$ 89.187.982,88 

PMBaC 
VABF - Aposentadoria: R$ 84.104.771,18 
VABF - Reversão em Pensão: R$ 6.570.971,81 
VABF - Invalidez: R$ 37.920,59 
VABF - Pecúlio: R$ 45.963,02 
--------------------------------------------- 
TOTAL VABF: R$ 90.759.626,61 

VACF (13x valor mensal): R$ 8.294.568,77 
VASF (13x valor mensal): R$ 97.583.162,03 

PMBaC Crédito Unitário: R$ 65.530.720,21 
PMBaC Custo Agregado: R$ 82.465.057,83


In [ ]:
# Caso deseje exportar para excel

Frase de interrupção para não precisar comentar cada linha

writer=pd.ExcelWriter(diretorio+'/ativos_v_comutacao.xlsx',engine='xlsxwriter')

ativos.to_excel(writer, index=False, sheet_name= 'ativos')
aposentados.to_excel(writer, index=False, sheet_name= 'aposentados')
pensionistas.to_excel(writer, index=False, sheet_name= 'pensionistas')
tabua_unica.to_excel(writer, index=False, sheet_name= 'tabua_unica')
tabua_masculina.to_excel(writer, index=False, sheet_name= 'tabua_masculina')
tabua_feminina.to_excel(writer, index=False, sheet_name= 'tabua_feminina')

workbook = writer.book
writer.close()